<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/chordanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pretty_midi
!git clone https://github.com/jmhuer/ModularSparseAutoencoder
!git clone https://github.com/music-x-lab/POP909-Dataset
!git clone https://github.com/jmhuer/HT
!git clone https://github.com/Tsung-Ping/functional-harmony
# %cd /content/POP909-Dataset/data_process
!pip install libfmp

     |████████████████████████████████| 5.6 MB 14.6 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=59b7f9bf648b7b88f6dae57bee6469579b1108f84e2f2450d25a70ce7eb9c78b
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
Cloning into 'ModularSparseAutoencoder'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 10), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (25/25), done.
Cloning into 'POP909-Dataset'...
remote: Enumerating objects: 9265, done.
remote: Counting objects: 100% (9265/9265), done.
remote: Compressing objects: 100% (8157/8157), done.
remote: Total 9265 (delta 13), reused 9245 (delta 6), pack-reused 0
Receiving objects: 100% (9265/9265), 45.75 MiB | 23.98 MiB/s,

In [2]:
#@title Pytorch for DL

import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import torch
from torch.nn.utils import weight_norm
import numpy as np
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)


def get_model_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

Using device:  cuda


# This preprocesses data and creates pickle


In [3]:

# !pip install tensorflow-gpu==1.15
from HT.chord_symbol_recognition import train_HT
from collections import Counter, namedtuple


# Chord symbol recognition
# train_BTC() # Bi-directional Transformer for Chord Recognition
# train_CRNN() # Convolutional Recurrent Neural Network
root_dict = {'C': 0, 'C+': 1, 'D': 2, 'D+': 3, 'E': 4, 'F': 5, 'F+': 6, 'G': 7, 'G+': 8, 'A': 9, 'A+': 10, 'B': 11, 'pad': 12}
tquality_dict = {'M': 0, 'm': 1, 'O': 2, 'pad': 3}  # 'O' stands for 'others'
n_chord_classes = 24 + 1  # 24 major-minor modes plus 1 others


# Hyperparameters
hyperparameters = namedtuple('hyperparameters',
                              ['dataset',
                              'test_set_id',
                              'graph_location',
                              'n_root_classes',
                              'n_tquality_classes',
                              'n_chord_classes',
                              'n_steps',
                              'input_embed_size',
                              'n_layers',
                              'n_heads',
                              'train_sequence_with_overlap',
                              'initial_learning_rate',
                              'drop',
                              'n_batches',
                              'n_training_steps',
                              'n_in_succession',
                              'annealing_rate'])

hp = hyperparameters(dataset='/content/', # {'BPS_FH', 'Preludes'}
                      test_set_id=1, # {1, 2, 3, 4}
                      graph_location='model',
                      n_root_classes=len(root_dict.keys()),
                      n_tquality_classes=len(tquality_dict.keys()),
                      n_chord_classes=n_chord_classes,
                      n_steps=128,
                      input_embed_size=128,
                      n_layers=2,
                      n_heads=4,
                      train_sequence_with_overlap=True,
                      initial_learning_rate=1e-4,
                      drop=0.1,
                      n_batches=40,
                      n_training_steps=100000,
                      n_in_succession=10,
                      annealing_rate=1.1)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from HT.BPS_FH_preprocessing import main

main()

Message: load note data ...


# Make pytorch dataloader

In [ ]:
# !pip install tensorflow-gpu==1.15
from HT.chord_symbol_recognition import train_HT
from collections import Counter, namedtuple
from HT.BPS_FH_preprocessing import main
from HT.chord_symbol_recognition import load_data_symbol
#

import pickle
import os
import sys
# import utils
import pretty_midi 
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch


class BPS(Dataset):
    def __init__(self, data):
        "todo: transpose"
        self.all_data = []
        temp = []
        self.label = []
        temp2 = []
        self.chord_change = []
        temp3 = []      
        for pi,chord, cc in zip(data["pianoroll"],data["tchord"],data['label']['chord_change']):
            temp.append(torch.tensor(pi).double())
            temp2.append(torch.tensor(chord).double())
            temp3.append(torch.tensor(cc).double())
        self.all_data = torch.cat(temp)
        self.label = torch.cat(temp2)
        self.chord_change = torch.cat(temp3)
        print("data" , self.all_data.shape)
        print("label" , self.label.shape)
        ##time.sleep(120)
        self.transform = self.make_transform()
    def __len__(self):
        return len(self.all_data)
    def __getitem__(self, idx):
        piano_roll_slice = self.all_data[idx,:]
        label_slice = self.label[idx]
        chord_change_slice = self.chord_change[idx]
        tranformed_piano_roll_slice = self.transform["norm"](piano_roll_slice[None][None])[0,0,:]
        #print(tranformed_piano_roll_slice.shape)
        #print("label_slice", label_slice)
        return tranformed_piano_roll_slice, label_slice, chord_change_slice
    def make_transform(self):
        mean = self.all_data.mean()
        std = self.all_data.std() * 2
        # print("mean ", mean.shape)
        # print("std ", std.shape)
        tensor_transform = {
          'norm':
              transforms.Compose([
                  transforms.Normalize([mean], [std])  # Imagenet standards
              ]),
          "inverse_norm":
                transforms.Normalize(
                  mean= [-m/s for m, s in zip([mean], [std])],
                  std= [1/s for s in [std]]
                )
            }
        return tensor_transform


def load_data(train_data, test_data,  num_workers=0, batch_size=16, random_seed = 40):
    '''
    this data loading proccedure assumes dataset/train/ dataset/val/ folders
    also assumes transform dictionary with train and val
    '''
    dataset_train = BPS(train_data) 
    dataset_val = BPS(test_data) 

    print("Size of train dataset: ",len(dataset_train))
    print("Size of val dataset: ",len(dataset_val))

    dataloaders = {
        'train': DataLoader(dataset_train, batch_size=batch_size, shuffle=False, drop_last=True),
        'val': DataLoader(dataset_val, batch_size=batch_size, shuffle=False, drop_last=True)
    }
    return dataloaders



##only tensor transforms



train_data, test_data = load_data_symbol(dir=hp.dataset + 'BPS_FH_preprocessed_data_MIREX_Mm.pickle', test_set_id=hp.test_set_id, sequence_with_overlap=hp.train_sequence_with_overlap)
print("load_data_symbol train_data size: {}".format(train_data["tchord"].shape))



dataset = load_data(train_data, test_data, batch_size=16)



#play example from BPS dataset

In [ ]:


def piano_roll_to_pretty_midi(piano_roll, fs=8, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm




In [ ]:
import IPython.display
index = 45 ## 44 66 & 0 & 1 500 omg 1021

#lets play a batch 
tr = dataset["val"].dataset.transform["inverse_norm"]
listen = []
for i,ba in enumerate(dataset["val"]): 
    ##print(ba[0].shape)
    if i < index: 
        continue
    listen.append(tr(ba[0][:,None,None,:]))
    if len(listen)==10: 
        piano_roll = torch.cat(listen)[:,0,0,:].T
        break

def pad88to128(piano_roll):
    arr = np.zeros((128,int(piano_roll.shape[1])))
    pad = (128 - 88)//2
    arr[pad:(128-pad),0:arr.shape[1]] = piano_roll
    return arr

arr = pad88to128(piano_roll)
pm = piano_roll_to_pretty_midi(arr)
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)


In [ ]:
import argparse
import datetime
import json
import os

import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim




def get_args():
    parser = argparse.ArgumentParser()

    # Architecture Flags
    parser.add_argument('--intermediate_dim', type=int, default=32)
    parser.add_argument('--stripe_dim', type=int, default=32)
    parser.add_argument('--num_stripes', type=int, default=10)
    parser.add_argument('--num_active_neurons', type=int, default=4)
    parser.add_argument('--num_active_stripes', type=int, default=3)
    parser.add_argument('--layer_sparsity_mode', type=str, default='lifetime')  # Set to none, ordinary, boosted, or lifetime.
    parser.add_argument('--stripe_sparsity_mode', type=str, default='routing')  # Set to none, ordinary, or routing.
    parser.add_argument('--distort_prob', type=float, default=.01)  # Probability of stripe sparsity mask bits randomly flipping.
    parser.add_argument('--distort_prob_decay', type=float, default=.025)  # Lowers distort_prob by this amount every epoch.

    # Boosting Flags - Only necessary when layer_sparsity_mode is set to boosted.
    parser.add_argument('--alpha', type=float, default=.8)
    parser.add_argument('--beta', type=float, default=1.2)

    # Routing Flags - Only necessary when stripe_sparsity_mode is set to routing.
    parser.add_argument('--routing_l1_regularization', type=float, default=0.1)
    parser.add_argument('--log_average_routing_scores', type=bool, default=True)

    # Lifetime Stripe Flag - Only necessary when stripe_sparsity_mode is set to lifetime.
    # Within a stripe, this specifies the proportion of samples that may activate the stripe.
    parser.add_argument('--active_stripes_per_batch', type=float, default=4)

    # Training Flags
    parser.add_argument('--lr', type=float, default=.0001)
    parser.add_argument('--momentum', type=float, default=.1)
    parser.add_argument('--num_epochs', type=int, default=4)
    parser.add_argument('--batch_size', type=int, default=8)
    parser.add_argument('--data_path', type=str, default='data.csv')
    parser.add_argument('--log_path', type=str, default='logs')
    parser.add_argument('--log_class_specific_losses', type=bool, default=False)
    parser.add_argument('--log_average_activations', type=bool, default=True)
    parser.add_argument('--use_cuda_if_available', type=bool, default=True)

    args, unknown = parser.parse_known_args()
    return vars(args)
args = get_args()
print(args["intermediate_dim"])

In [ ]:
from ModularSparseAutoencoder.model import Net
from ModularSparseAutoencoder.train import train


num_stripes = args['num_stripes']
num_epochs = args['num_epochs']
batch_size = args['batch_size']

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

net = Net(args['intermediate_dim'],
          args['stripe_dim'],
          args['num_stripes'],
          args['num_active_neurons'],
          args['num_active_stripes'],
          args['layer_sparsity_mode'],
          args['stripe_sparsity_mode'],
          args['distort_prob'],
          args['alpha'],
          args['beta'],
          args['active_stripes_per_batch'],
          device).double().to(device)

# net.load_state_dict(torch.load("/content/net.pth"))

criterion = nn.MSELoss().to(device)
# optimizer = optim.SGD(net.parameters(),
#                       lr=args['lr'],
#                       momentum=args['momentum'])
optimizer = optim.Adam(net.parameters(), lr=args['lr'],  
                       betas=(0.8, 0.999), eps=1e-08, weight_decay=0, amsgrad=True) ##this has weight decay just like you implemented

timestamp = str(datetime.datetime.now()).replace(' ', '_')
root_path = os.path.join(args['log_path'],
                          args['layer_sparsity_mode'],
                          args['stripe_sparsity_mode'],
                          timestamp)
print(f'Logging results to path:  {root_path}')

distort_prob_decay = args['distort_prob_decay']
routing_l1_regularization = (args['routing_l1_regularization'] if args['stripe_sparsity_mode'] == 'routing' else 0)
log_class_specific_losses = args['log_class_specific_losses']
should_log_average_routing_scores = (
            args['stripe_sparsity_mode'] == 'routing' and args['log_average_routing_scores'])

train(net,
      criterion,
      optimizer,
      root_path,
      dataset,
      num_stripes,
      num_epochs,
      distort_prob_decay,
      routing_l1_regularization,
      log_class_specific_losses,
      should_log_average_routing_scores)
  
# if args['log_average_activations']:
#     average_activations_path = os.path.join(root_path, 'average_activations.json')
#     with open(average_activations_path, 'w') as f:
#         average_activations = net.get_average_activations(X_test, Y_test, device=device).tolist()
#         f.write(json.dumps(average_activations))

flag_values_path = os.path.join(root_path, 'experiment_config.json')
with open(flag_values_path, 'w') as f:
    f.write(json.dumps(args))

#evaluate 

#re init data for batch 128

In [ ]:
import IPython.display
index = 45 ## 44 66 & 0 & 1 500 omg 1021

#lets play a batch 
tr = dataset["val"].dataset.transform["inverse_norm"]
listen = []
for i, ba in enumerate(dataset["val"]): 
    if i < index: 
        continue
    x_var = ba[0].to(net.device)
    xpred_var = net(x_var)[:,None,None,:]
    listen.append(tr(xpred_var))
    if len(listen)==10: 
        piano_roll = torch.cat(listen)[:,0,0,:].T
        piano_roll = (piano_roll>=1) 
        break

def pad88to128(piano_roll):
    arr = np.zeros((128,int(piano_roll.shape[1])))
    pad = (128 - 88)//2
    arr[pad:(128-pad),0:arr.shape[1]] = piano_roll
    return arr

arr = pad88to128(piano_roll.cpu().detach().numpy())
pm = piano_roll_to_pretty_midi(arr)
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)

In [ ]:
import libfmp.c1
score = libfmp.c1.midi_to_list(pm)

libfmp.c1.visualize_piano_roll(score, figsize=(8, 3))


In [ ]:
import libfmp.c1
score = libfmp.c1.midi_to_list(pm)

libfmp.c1.visualize_piano_roll(score, figsize=(8, 3))



In [ ]:
#load data with batch size to be used in transformer 
traintr = dataset["train"].dataset.transform["inverse_norm"]
valtr = dataset["val"].dataset.transform["inverse_norm"]


preprocessed_train_data = []
preprocessed_train_labeldata = []
preprocessed_train_cc = []

for i, batch in enumerate(dataset["train"]): 
    x_var = batch[0].to(net.device)
    x_var = x_var.to(net.device)
    xpred_var = net(x_var)
    # xpred_var = x_var
    preprocessed_train_data.append(traintr(xpred_var[:,None,None,:])[:,0,0,:].cpu().detach().numpy())
    preprocessed_train_labeldata.append(batch[1].cpu().detach().numpy())
    preprocessed_train_cc.append(batch[2].cpu().detach().numpy())

preprocessed_test_data = []
preprocessed_test_labeldata = []
preprocessed_test_cc = []

for i, batch in enumerate(dataset["val"]): 
    x_var = batch[0].to(net.device)
    x_var = x_var.to(net.device)
    xpred_var = net(x_var)
    # xpred_var = x_var
    preprocessed_test_data.append(valtr(xpred_var[:,None,None,:])[:,0,0,:].cpu().detach().numpy())
    preprocessed_test_labeldata.append(batch[1].cpu().detach().numpy())
    preprocessed_test_cc.append(batch[2].cpu().detach().numpy())


preprocessed_train_data = np.array(preprocessed_train_data)
preprocessed_train_labeldata= np.array(preprocessed_train_labeldata)
preprocessed_train_cc = np.array(preprocessed_train_cc)

preprocessed_test_data = np.array(preprocessed_test_data)
preprocessed_test_labeldata = np.array(preprocessed_test_labeldata)
preprocessed_test_cc = np.array(preprocessed_test_cc)

print(preprocessed_train_data.shape)
print(preprocessed_test_data.shape)


print(preprocessed_train_labeldata.shape)
print(preprocessed_test_labeldata.shape)

print(preprocessed_train_cc.shape)
print(preprocessed_test_cc.shape)


In [ ]:
import gc
del dataset
gc.collect()

train_data2 = {}
test_data2 = {}
train_data2['label'] = {}
test_data2['label'] = {}

train_data2['label']['chord_change']= preprocessed_train_cc
train_data2["pianoroll"] =  preprocessed_train_data  
train_data2["tchord"] =  preprocessed_train_labeldata  

test_data2['label']['chord_change'] =  preprocessed_test_cc
test_data2["pianoroll"] =  preprocessed_test_data  
test_data2["tchord"] =  preprocessed_test_labeldata  

dataset = load_data(train_data2, test_data2, batch_size=128)


In [ ]:
#load data with batch size to be used in transformer 
traintr = dataset["train"].dataset.transform["inverse_norm"]
valtr = dataset["val"].dataset.transform["inverse_norm"]

#all
how_many_batches_train = 54320 
how_many_batches_test = 294

preprocessed_train_data = []
preprocessed_train_labeldata = []
preprocessed_train_cc = []

for i, batch in enumerate(dataset["train"]): 
    x_var = batch[0]
    x_var = x_var
    xpred_var = x_var
    preprocessed_train_data.append(traintr(xpred_var[:,None,None,:])[:,0,0,:].detach().numpy())
    preprocessed_train_labeldata.append(batch[1].detach().numpy())
    preprocessed_train_cc.append(batch[2].detach().numpy())
    if len(preprocessed_train_data)==how_many_batches_train: break



preprocessed_test_data = []
preprocessed_test_labeldata = []
preprocessed_test_cc = []

for i, batch in enumerate(dataset["val"]): 
    x_var = batch[0]
    x_var = x_var
    xpred_var = x_var
    preprocessed_test_data.append(valtr(xpred_var[:,None,None,:])[:,0,0,:].detach().numpy())
    preprocessed_test_labeldata.append(batch[1].detach().numpy())
    preprocessed_test_cc.append(batch[2].detach().numpy())
    if len(preprocessed_test_data)==how_many_batches_test: break


preprocessed_train_data = np.array(preprocessed_train_data)
preprocessed_train_labeldata= np.array(preprocessed_train_labeldata)
preprocessed_train_cc = np.array(preprocessed_train_cc)

preprocessed_test_data = np.array(preprocessed_test_data)
preprocessed_test_labeldata = np.array(preprocessed_test_labeldata)
preprocessed_test_cc = np.array(preprocessed_test_cc)






print(preprocessed_train_data.shape)
print(preprocessed_test_data.shape)


print(preprocessed_train_labeldata.shape)
print(preprocessed_test_labeldata.shape)

print(preprocessed_train_cc.shape)
print(preprocessed_test_cc.shape)


# this trains Harmony transformer 

In [ ]:
from HT.chord_symbol_recognition import train_HT
from collections import Counter, namedtuple
# Chord symbol recognition
# train_BTC() # Bi-directional Transformer for Chord Recognition
# train_CRNN() # Convolutional Recurrent Neural Network
train_data3 = {}
test_data3 = {}
train_data3['label'] = {}
test_data3['label'] = {}

train_data3['label']['chord_change']= preprocessed_train_cc
train_data3["len"] = np.array([128 for i in range(how_many_batches_train)])
train_data3["pianoroll"] =  preprocessed_train_data  
train_data3["tchord"] =  preprocessed_train_labeldata  
train_data3['root'] = np.zeros((how_many_batches_train,128))
train_data3['tquality'] =  np.zeros((how_many_batches_train,128))

test_data3['label']['chord_change'] =  preprocessed_test_cc
test_data3["len"] = np.array([128 for i in range(how_many_batches_test)])
test_data3["pianoroll"] =  preprocessed_test_data  
test_data3["tchord"] =  preprocessed_test_labeldata  
test_data3['root'] =  np.zeros((how_many_batches_test,128))
test_data3['tquality'] =  np.zeros((how_many_batches_test,128))

# import os
train_HT(hp, train_data3, test_data3)

# print(test_data["pianoroll"].shape, test_data2["pianoroll"].shape)
# print(test_data['label']['chord_change'].shape, test_data2['label']['chord_change'].shape)
# print(test_data["tchord"].shape, test_data2["tchord"].shape)
# print(test_data["len"].shape, test_data2["len"].shape)